# Task

- Створити процес, для повороту документів в книжкову орієнтацію, з використанням ML моделі яка буде визначати кут від -30 до 30 градусів.
- Дозволено використовувати pytorch і tf та будь які бібліотеки, які будуть допоміжними.
- По часовим обмеженням: модель має працювати орієнтовно 5 секунд або менше, але це не критична вимога.
- Процес має бути у презентативному стані, тобто в міру задокументованим і зрозумілим при ревю.

# Angle detection using open-cv

In [1]:
import numpy as np
import cv2
import math
from PIL import Image
import json
import os 
from natsort import natsorted # natural sorting

In [2]:
def calculate_angle(image_path):

    img_before = cv2.imread(image_path)

    img_gray = cv2.cvtColor(img_before, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.Canny(img_gray, 100, 100, apertureSize=3)
    lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)

    angles = []

    for [[x1, y1, x2, y2]] in lines:
        cv2.line(img_before, (x1, y1), (x2, y2), (255, 0, 0), 3)
        angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
        angles.append(angle)

    median_angle = np.median(angles)


    return round(median_angle, 2)

In [3]:
PREDICTED_ANGLES = {}
PATH = 'invoices_rotated'

for image in natsorted(os.listdir(PATH)):
    image_path = os.path.join(PATH, image)

    # the negative angle is used because we need not a rotation angle 
    # but an angle-to-rotate the image to normalize it
    negative_angle = -(round(calculate_angle(image_path)))

    # detect an angle within the range of [-30,30] degrees
    if negative_angle in range(-30,31):
        PREDICTED_ANGLES[image_path] = negative_angle
        print(f'{image_path}: {negative_angle}')
    else:
        continue

with open('predicted_angles.json', 'w') as json_file:
    # store image paths as keys and predicted angles as values into a  JSON file
    json.dump(PREDICTED_ANGLES, json_file, indent=4)

invoices_rotated/0.png: 7
invoices_rotated/1.png: -2
invoices_rotated/2.png: -9
invoices_rotated/3.png: -7
invoices_rotated/4.png: -1
invoices_rotated/5.png: -6
invoices_rotated/6.png: 5
invoices_rotated/7.png: -2
invoices_rotated/8.png: -4
invoices_rotated/9.png: -1
invoices_rotated/10.png: -4


### Image rotation using the calculated angle value

In [4]:
# specify the image path
image_path = 'invoices_rotated/10.png'

# calculate angle using before defined function
detected_angle = calculate_angle(image_path)

# create at Image object
pil_image = Image.open(image_path)

# rotate image by the calculated angle
rotated_image = pil_image.rotate(detected_angle)

# show the rotated image
rotated_image.show()